# Water models
Here we will demonstrate how MoSDeF can be used to compare two different force fields; in this case, two atomistic water models

In [ ]:
import mbuild as mb
from foyer import Forcefield

In [ ]:
water = mb.load('O', smiles=True)
water.name = 'water'

In [ ]:
system = mb.fill_box(compound=water, n_compounds=1000, density=993)

In [ ]:
system.to_parmed(residues=['water'])

In [ ]:
SPCE = Forcefield('spce.xml')
TIP3P = Forcefield('tip3p.xml')

In [ ]:
system_spce = SPCE.apply(system.to_parmed(residues=['water']))
system_tip3p = TIP3P.apply(system.to_parmed(residues=['water']))

In [ ]:
system_spce.save('water_spce.gro')
system_spce.save('water_spce.top')
system_tip3p.save('water_tip3p.gro')
system_tip3p.save('water_tip3p.top')

In [ ]:
!gmx grompp -c water_spce.gro -p water_spce.top -f npt.mdp -o water_spce.tpr

In [ ]:
!gmx mdrun -v -deffnm water_spce

In [ ]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid.inset_locator import InsetPosition

In [ ]:
trj = md.load('water_spce.xtc', top='water_spce.gro')

In [ ]:
!echo 4 | gmx energy -f water_spce.edr

In [ ]:
from mpl_toolkits.axes_grid.inset_locator import InsetPosition

data = np.loadtxt('energy.xvg', comments=['#', '@'])

fig, ax = plt.subplots()

ax.plot(data[:, 0], data[:, 1])
axins = plt.axes([0,0,1,1])

ip = InsetPosition(ax, [0.2,0.2,0.7,0.7])
axins.set_axes_locator(ip)

axins.plot(data[100:, 0], data[100:, 1])